In [ ]:
pip install requests

In [ ]:
pip install selenium

In [ ]:
pip install beautifulsoup4

In [1]:
# import necessary modules
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from selenium import webdriver
from tqdm import tqdm
import time
import datetime
import random
import urllib.request 
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import os
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
# browser.quit() # close the whole browser
# browser.close() # close the tab

In [ ]:
#URL = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html'
URL = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96'
#URL = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=24'
'''
page = requests.get(URL)
driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
driver.get(URL)
soup = BeautifulSoup(driver.page_source, 'html.parser')
print(soup.prettify())

product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})
print(len(product_cards))

i = 0
for product_card in product_cards:
    i += 1
    product_link = product_card.find('a')['href']
    image = product_card.find('div', {'class' : "BrowseProductCard-imageWrap"})
    image_link = image.find('img')['src']

    print(f"Image link here: {image_link}\n")
    print(f"Link to product {i} here: {product_link}\n")
'''

In [8]:
def get_product_info_from_mainpage(URL):
    product_links = []
    image_links = []
    prices = []
    
    #page = requests.get(URL)
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    try:
        driver.get(URL)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.randint(3,3))
        driver.implicitly_wait(20)
        product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})
        '''
        while len(product_cards) != 96:
            product_cards = soup.find_all('div', {"class" : "ProductCard-container ProductCard-container--contained"})
            #wait.util(len(product_cards) == 96)
        '''
        print(len(product_cards))
        i = 0
        for product_card in product_cards:
            try:
                i += 1
                #get product link
                product_link = product_card.find('a')['href']
                product_links.append(product_link)

                #get image link
                image = product_card.find('div', {"class" : "BrowseProductCard-imageWrap"})
                image_link = image.find('img')['src']
                image_links.append(image_link)

                #get price
                price_block = product_card.find('div', {"class" : "ProductCard-pricing"})
                #price = price_block.findElement(By.xpath("//*[contains(@class,'ProductCard-price')]"));
                #price = price_block.find_element_by_css_selector("span[class^='ProductCard-price']")
                price = price_block.find('span').text


                '''
                try:
                    price = price_block.find('span', {"class" : "ProductCard-price"}).text
                except:
                    price = price_block.find('span', {"class" : "ProductCard-price ProductCard-price--sale"}).text
                '''
                prices.append(price)
                print(price)


                #print info
                #print(f"Image link here: {image_link}\n")
                #print(f"Link to product {i} here: {product_link}\n")
            except:
                print("error")
                pass
        driver.close()
        return (product_links, image_links, prices)
    except TimeoutException as e:
        print("Page load Timeout Occured. Quiting !!!")
        driver.close()
    #browser.switch_to.window(browser.window_handles[0])
    #return (product_links, image_links, prices)

def get_info_from_productlink(product_link):  #get the first image
    info = dict()
    page = requests.get(product_link)
    driver = webdriver.Chrome('C:/Users/Brian/Downloads/chromedriver_win32/chromedriver.exe')
    driver.get(product_link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(random.randint(3,3))

    #price_block = soup.find('div', {"class" : "BasePriceBlock BasePriceBlock--highlight"})
    #price_block = soup.find('div', {"class" : "BasePriceBlock"})
    price_block = soup.find('span', {"class" : "notranslate"})
    price = price_block.text
    info['price'] = price
    #print(price)   
    
    image = soup.find('div', {'class' : "ImageComponent ImageComponent--overlay"})
    image_link = image.find('img')['src']
    info['img_url'] = image_link
    
    #print(f"Image link: {image_link}\n")
    #print(f"Link to product: {product_link}\n")

    driver.close()
    #browser.switch_to.window(browser.window_handles[0])
    return info
    

In [9]:
get_product_info_from_mainpage('https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96&curpage=1')

96
$8,399.99
$2,539.99
$1,029.99
$1,349.99
$1,759.99
$1,369.99
$1,329.99
$1,089.99
$1,079.99
$1,459.99
$1,099.99
$1,169.99
$1,389.99
$1,599.99
$1,019.99
$1,349.00
$1,329.99
$1,059.99
$1,499.99
$1,699.99
$1,069.99
$1,229.99
$4,199.99
$1,009.99
$1,019.99
$1,199.99
$1,019.99
$2,539.99
$1,199.99
$1,119.99
$1,229.17
$2,699.99
$1,569.99
$1,059.99
$1,199.00
$1,749.99
$2,699.99
$1,049.99
$1,069.99
$1,199.99
$1,159.99
$2,299.99
$1,028.32
$1,099.99
$1,049.00
$1,339.99
$1,089.99
$2,299.99
$1,629.99
$1,349.99
$1,259.99
$2,599.99
$1,189.99
$2,369.99
$1,129.99
$1,149.99
$1,219.99
$1,079.99
$1,069.99
$1,139.99
$1,139.99
$1,139.99
$1,049.00
$1,039.99
$1,469.99
$1,079.99
$1,059.99
$1,299.99
$1,539.99
$1,139.99
$1,099.99
$1,029.99
$1,099.99
$2,399.99
$1,499.99
$1,969.99
$1,439.99
$1,339.99
$1,069.99
$1,299.99
$1,099.99
$1,769.99
$1,299.00
$1,059.99
$1,339.99
$1,079.99
$1,179.00
$2,899.99
$1,999.99
$2,159.99
$5,400.00
$1,229.99
$1,139.99
$3,999.99
$1,169.99
$1,359.99


(['https://www.wayfair.com/furniture/pdp/eichholtz-taylor-sofa-eitz3452.html?piid=',
  'https://www.wayfair.com/furniture/pdp/corrigan-studio-renae-genuine-leather-89-tuxedo-arm-sofa-w000360190.html?piid=2064628216&categoryid=413892&placement=1&slot=0&sponsoredid=2d1105fb3d5fd54f3d9057253a9f7dd1cd7d0d935c3efc07e6507b51a0db8930&_txid=I%2BF9Ol8WULwqXw%2F8zwVRAg%3D%3D&isB2b=0&auctionId=a8509ac1-83a6-40d2-a87f-f43a1a4d5271',
  'https://www.wayfair.com/furniture/pdp/latitude-run-sauer-96-rolled-arm-sofa-w001985188.html?piid=',
  'https://www.wayfair.com/furniture/pdp/trent-austin-design-harlem-genuine-leather-chesterfield-86-rolled-arm-sofa-trnt3405.html?piid=34237218',
  'https://www.wayfair.com/furniture/pdp/three-posts-brinson-genuine-leather-chesterfield-95-rolled-arm-sofa-w001243560.html?piid=1221870775',
  'https://www.wayfair.com/furniture/pdp/foundstone-idris-80-square-arm-sofa-w002004276.html?piid=',
  'https://www.wayfair.com/furniture/pdp/breakwater-bay-oliver-90-reversible-modul

In [ ]:
get_info_from_productlink('https://www.wayfair.com/furniture/pdp/three-posts-brinson-genuine-leather-chesterfield-95-rolled-arm-sofa-w001243560.html?piid=1221870775')

In [10]:
def save_data(product_info_list, i):
    result = [[product[0], product[1]['price'], product[1]['img_url']] for product in product_info_list]
    df = pd.DataFrame(result)
    now = str(datetime.datetime.now()).replace(" ","").replace(":","").replace("-","").split(".")[0][4:][:-2]
    df.to_csv(f'sofa_info_{str(i)}.csv', index=False, header=['item_no', 'price', 'img_url'])
    
def save_image_from_url(x, category):    
    item_no = x.name+1
    filename = f'./{category}/{item_no}.jpeg'
    url = x['img_url']
    try:
        urllib.request.urlretrieve(url, filename)
        return filename
    except:
        print('failed')
        return 'failed'
    
def save_images_from_csv(df):
    df['filename'] = df.progress_apply(lambda x: save_image_from_url(x, category = 'sofa'), axis=1)
    return df

In [ ]:
get_info_from_productlink("https://www.wayfair.com/furniture/pdp/eichholtz-taylor-sofa-eitz3452.html?piid=")

In [11]:
product_info = []
#base_url = 'https://www.wayfair.com/furniture/sb0/sofas-c413892.html?curpage='
base_url = 'https://www.wayfair.com/filters/furniture/sb1/sofas-c413892-p2479~1000~*.html?itemsperpage=96&curpage='
#base_url = 'https://www.wayfair.com/furniture/sb0/sofas-c413892.html?curpage='
#for page in tqdm(range(1,201)):

#data from pages 1 - 4
for page in tqdm(range(1,5)):
    url = base_url + str(page)
    #link_list = get_product_links_from_mainpage(url)  # latter will work even this is None
    (product_links, image_links, prices) = get_product_info_from_mainpage(url)
    print(f"Number of products on Page {page}: ", len(product_links))
    i = 5779
    for (product_url, img_url, price) in tqdm(zip(product_links, image_links, prices)):
        i += 1
        #item_no = f"{page}_{i}"
        item_no = f"{i}"
        try:
            info = {'price' : price, 'img_url' : img_url}
            product_info.append((item_no,info))     
        except:
            print(f"Failed. Product link: {product_url}")
            pass
save_data(product_info,1)
product_info

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

12
$8,399.99
$2,539.99
$1,029.99
$1,349.99
$1,759.99
$1,369.99
$1,329.99
$1,089.99
$1,079.99
$1,459.99
$1,099.99
$1,169.99
Number of products on Page 1:  12



12it [00:00, 26958.57it/s]
 25%|█████████████████████                                                               | 1/4 [00:19<00:59, 19.83s/it]

96
$1,059.99
$1,079.99
$1,069.99
$1,259.99
$1,389.99
$2,129.99
$1,049.99
$1,249.99
$1,039.99
$1,899.99
$1,089.99
$2,269.99
$1,389.99
$1,634.99
$1,329.99
$1,639.99
$1,129.99
$1,119.99
$1,249.99
$1,069.99
$1,129.99
$1,459.99
$1,756.00
$2,539.99
$1,499.00
$1,199.99
$1,279.99
$1,429.99
$1,079.99
$1,039.99
$1,529.99
$1,049.99
$1,029.99
$1,099.99
$3,699.99
$3,699.99
$1,389.99
$1,399.99
$1,239.99
$2,999.99
$1,049.99
$2,539.99
$1,693.56
$1,519.99
$2,369.99
$1,579.99
$1,049.99
$2,169.99
$1,299.99
$1,119.99
$1,039.99
$3,399.99
$1,329.99
$1,949.99
$1,229.99
$1,429.99
$1,199.99
$2,299.99
$1,029.99
$1,069.99
$8,399.99
$2,769.99
$1,059.99
$1,905.20
$2,599.99
$1,239.99
$1,049.99
$1,019.99
$1,649.99
$1,149.99
$1,069.99
$3,299.99
$1,798.00
$1,199.00
$1,089.99
$2,704.00
$1,229.99
$1,199.99
$3,059.99
$1,639.99
$1,079.99
$1,119.99
$1,199.99
$1,059.99
$2,396.86
$1,019.99
$1,089.99
$1,439.99
$1,179.99
$2,459.99
$1,119.99
$2,199.99
$3,399.99
$1,269.99
$1,029.99
$1,069.99
Number of products on Page 2:  96



96it [00:00, 119659.19it/s]
 50%|██████████████████████████████████████████                                          | 2/4 [00:45<00:43, 21.52s/it]

12
$4,999.99
$5,099.00
$1,339.99
$5,399.99
$1,659.99
$2,199.99
$1,199.99
$2,699.99
$2,799.99
$1,279.99
$2,439.99
$3,679.00
Number of products on Page 3:  12



12it [00:00, ?it/s]
 75%|███████████████████████████████████████████████████████████████                     | 3/4 [01:00<00:19, 19.59s/it]

12
$1,469.99
$3,399.99
$1,019.99
$1,299.99
$2,249.99
$1,299.99
$1,199.99
$1,049.99
$1,029.99
$1,309.00
$1,019.99
$1,099.99
Number of products on Page 4:  12



12it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:14<00:00, 18.57s/it]


[('5780',
  {'price': '$8,399.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/36770026/resize-h160-w160%5Ecompr-r85/1019/101965755/Taylor+Sofa.jpg'}),
 ('5781',
  {'price': '$2,539.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/89033325/resize-h160-w160%5Ecompr-r85/6687/66875866/Renae+Genuine+Leather+89%2522+Tuxedo+Arm+Sofa.jpg'}),
 ('5782',
  {'price': '$1,029.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/63333509/resize-h160-w160%5Ecompr-r85/6082/60825500/Sauer+96%2522+Rolled+Arm+Sofa.jpg'}),
 ('5783',
  {'price': '$1,349.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/04364329/resize-h160-w160%5Ecompr-r85/6128/61280951/Harlem+Genuine+Leather+Chesterfield+86%2522+Rolled+Arm+Sofa.jpg'}),
 ('5784',
  {'price': '$1,759.99',
   'img_url': 'https://secure.img1-fg.wfcdn.com/im/99369178/resize-h160-w160%5Ecompr-r85/7888/78886170/Brinson+Genuine+Leather+Chesterfield+95%2522+Rolled+Arm+Sofa.jpg'}),
 ('5785',
  {'price': '$1,369.99',
   'img_url': 'https://se

In [ ]:
df = pd.read_csv('sofa_info_1.csv')
print(df.shape)
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
df_new = save_images_from_csv(df)
df_new.to_csv('sofa.csv', index=False)

In [ ]:
df_new